In [ ]:
asdf

# Imports

In [2]:
import pathlib as pl
# import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import nesdis_gml_synergy.cloud_interface as ngsci
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

4097035

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# warnings.filterwarnings("default")

# download some data

In [ ]:
reload(nesdis_aws.nesdis_aws)
reload(nesdis_aws)

In [16]:
product = 'ACM' #FDC
path2folder_local=f'/export/htelg/tmp/ABI_L2_{product}/'
# pl.Path(path2folder_local).mkdir()
query = ngsci.AwsQuery(path2folder_local=path2folder_local,
                            satellite='16',
                            product=f'ABI-L2-{product}',
                            scan_sector='C',
                            # start='2020-09-30 18:00:00',
                            # end='2020-09-30 19:35:00',
                            start='2023-08-01 18:00:00',
                            # end='2018-11-01 00:00:00',
                            end='2023-08-01 18:05:00',
                            verbose = True
                           )


In [17]:
query.workplan

Get workplan:
workplan done


,path2file_aws,path2file_local
2023-08-01 18:01:17,noaa-goes16/ABI-L2-ACMC/2023/213/18/OR_ABI-L2-...,/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M6...


In [18]:
query.workplan.iloc[0].path2file_local

PosixPath('/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M6_G16_s20232131801174_e20232131803547_c20232131805009.nc')

In [ ]:
print(query.info_on_current_query())

In [ ]:
query.product_available_since

In [19]:
query.download()


[None]

# versions

There is a change from version M3 to M6. QF changes slightly but should not effect assesment. It looks like some flags are simply not used anymore??!

In [20]:
query.workplan.iloc[0].path2file_local.as_posix()

'/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M6_G16_s20232131801174_e20232131803547_c20232131805009.nc'

In [15]:
query.workplan.iloc[0].path2file_local.as_posix()

'/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M3_G16_s20182131802224_e20182131804597_c20182131805209.nc'

In [4]:
dsn = xr.open_dataset('/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M6_G16_s20232131801174_e20232131803547_c20232131805009.nc')
dso = xr.open_dataset('/export/htelg/tmp/ABI_L2_ACM/OR_ABI-L2-ACMC-M3_G16_s20182131802224_e20182131804597_c20182131805209.nc')

In [16]:
dsn.t.values, dso.t.values

(numpy.datetime64('2023-08-01T18:02:36.145581952'),
 numpy.datetime64('2020-01-01T18:02:37.015003008'))

In [5]:
dsn.DQF.attrs['flag_meanings'] 

'good_quality_qf bad_quality_qf space_qf spare spare spare degraded_quality_qf'

In [6]:
dso.DQF.attrs['flag_meanings']

'good_quality_qf invalid_due_to_not_geolocated_or_algorithm_non-execution_qf degraded_due_to_LZA_threshold_exceeded_qf invalid_due_to_bad_or_missing_input_11um_brightness_temperature_qf degraded_due_to_bad_input_3.9um_pixel_qf degraded_due_to_failed_0.64um_tests_qf degraded_due_to_other_bad_bands_qf'

In [10]:
dso.DQF.attrs['flag_meanings'] == 'good_quality_qf invalid_due_to_not_geolocated_or_algorithm_non-execution_qf degraded_due_to_LZA_threshold_exceeded_qf invalid_due_to_bad_or_missing_input_11um_brightness_temperature_qf degraded_due_to_bad_input_3.9um_pixel_qf degraded_due_to_failed_0.64um_tests_qf degraded_due_to_other_bad_bands_qf'


True

In [7]:
dsn.DQF.attrs['flag_meanings'] == dso.DQF.attrs['flag_meanings']

False

In [8]:
dsn.DQF.attrs['flag_meanings'] == 'good_quality_qf bad_quality_qf space_qf spare spare spare degraded_quality_qf'


True

In [9]:
dsn.DQF.attrs['flag_meanings'].split()

['good_quality_qf',
 'bad_quality_qf',
 'space_qf',
 'spare',
 'spare',
 'spare',
 'degraded_quality_qf']

# open

In [7]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
stations = surfrad.network.stations

In [56]:
reload(ngs)

<module 'nesdis_gml_synergy.satlab' from '/export/htelg/prog/nesdis_gml_synergy/nesdis_gml_synergy/satlab.py'>

In [59]:
p2fld = pl.Path('/nfs/stu3data2/Satellite_data/goes/16/ACM/raw/')
fns = list(p2fld.glob('*'))

In [70]:
for fn in fns: 
    fn.unlink()

In [67]:
# fn = '/mnt/telg/data/nesdis/goes/ABI_L2_ADP/OR_ABI-L2-ADPC-M6_G16_s20211431801169_e20211431803542_c20211431804531.nc' 
# fn = '/nfs/stu3data2/Satellite_data/goes/16/ACM/raw/OR_ABI-L2-ACMC-M6_G16_s20231142131172_e20231142133545_c20231142134496.nc'
sat_ds = ngs.open_file(fns[3], verbose=True)
self = sat_ds
self.ds.compute()

product name: ABI-L2-ACMC-M6
identified as: ABI_L2_ACM.


RuntimeError: NetCDF: HDF error

In [41]:
proj = sat_ds.project_on_sites(stations)

In [14]:
self.ds.DQF

<xarray.DataArray 'DQF' (y: 1500, x: 2500)>
array([[nan, nan, nan, ...,  0.,  0.,  0.],
       [nan, nan, nan, ...,  0.,  0.,  0.],
       [nan, nan, nan, ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)
Coordinates:
    t                                datetime64[ns] ...
  * y                                (y) float64 0.1282 0.1282 ... 0.04427
  * x                                (x) float64 -0.1013 -0.1013 ... 0.03861
    y_image                          float32 ...
    x_image                          float32 ...
    retrieval_local_zenith_angle     float32 ...
    quantitative_local_zenith_angle  float32 ...
    retrieval_solar_zenith_angle     float32 ...
Attributes: (12/13)
    long_name:                    ABI L2+ Clear Sky Mask data quality flags
    standard_name:                status_flag
    valid_range:                  [0 6]
    units:                        1
    grid_mapping:                 goes_imager_projection
    cell_methods:                 retrieval_local_zenith_angle: point quantit...
    ...                           ...
    flag_meanings:                good_quality_qf bad_quality_qf space_qf spa...
    number_of_qf_values:          7
    percent_good_quality_qf:      1.0
    percent_bad_quality_qf:       0.0
    percent_space_qf:             0.0
    percent_degraded_quality_qf:  0.0

In [15]:
self.qf_managment.qf_by_variable

{'BCM': {'high': [0], 'medium': [6], 'bad': [1, 2, 3, 4, 5]},
 'ACM': {'high': [0], 'medium': [6], 'bad': [1, 2, 3, 4, 5]},
 'Cloud_Probabilities': {'high': [0], 'medium': [6], 'bad': [1, 2, 3, 4, 5]}}

In [47]:
var = self.tp_var

In [48]:
valid_qf = self.tp_valid_qf

In [44]:
self.ds.DQF.isin(valid_qf)

<xarray.DataArray 'DQF' (y: 1500, x: 2500)>
array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])
Coordinates:
    t                                datetime64[ns] ...
  * y                                (y) float64 0.1282 0.1282 ... 0.04427
  * x                                (x) float64 -0.1013 -0.1013 ... 0.03861
    y_image                          float32 ...
    x_image                          float32 ...
    retrieval_local_zenith_angle     float32 ...
    quantitative_local_zenith_angle  float32 ...
    retrieval_solar_zenith_angle     float32 ...

In [50]:
self.ds[var]#.where(self.ds.DQF.isin(valid_qf))

<xarray.DataArray 'Cloud_Probabilities' (y: 1500, x: 2500)>
[3750000 values with dtype=float32]
Coordinates:
    t                                datetime64[ns] ...
  * y                                (y) float64 0.1282 0.1282 ... 0.04427
  * x                                (x) float64 -0.1013 -0.1013 ... 0.03861
    y_image                          float32 ...
    x_image                          float32 ...
    retrieval_local_zenith_angle     float32 ...
    quantitative_local_zenith_angle  float32 ...
    retrieval_solar_zenith_angle     float32 ...
Attributes:
    long_name:     ABI Cloud Mask product Cloud Probabilities
    grid_mapping:  goes_imager_projection
    valid_range:   [0 1]

In [49]:
self.ds[var].where(self.ds.DQF.isin(valid_qf))

RuntimeError: NetCDF: HDF error

In [42]:
proj.projection2area.sel(site = 'BND', stats = 'mean', radius = 50, 
                         # data_quality = 'good'
                        )

RuntimeError: NetCDF: HDF error

In [ ]:
proj.projection2point

# Plot on map

In [ ]:
scale = 0.5
width=2800000 *scale
height=1700000 * scale
a, bmap = surfrad.network.stations.Table_Mountain.plot(width=width, height=height)
bmap, pc, cb = sat_ds.plot('Temp', bmap = bmap)
bmap.drawcounties()
# f = plt.gcf()
# pc.set_clim(vmax= 10)

# Download and project

In [5]:
import pathlib as pl
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad
import os
os.getpid()

7468

In [6]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as surfrad

stations = surfrad.network.stations

In [ ]:
# def function(row, stations):#, alt_var = None):
#     # if isinstance(alt_var, type(None)):
#     #     alt_var = variable
#     # read the file
#     ngsinst = ngs.open_file(row.path2file_local)
    
#     # project to stations
#     projection = ngsinst.project_on_sites(stations)

#     # merge closest gridpoint and area
#     point = projection.projection2point.copy()#.sel(site = 'TBL')
#     point['DQF'] = point.DQF.astype(int) # for some reason this was float64... because there are some nans in there
    
#     # change var names to distinguish from area
#     for var in ngsinst.valid_2D_variables:
#         point = point.rename({var: f'{var}_on_pixel',})
#     point = point.rename({'DQF': 'DQF_on_pixel'})
    
#     # merge aerea and point
#     ds = projection.projection2area.merge(point)#.rename({alt_var: f'{alt_var}_on_pixel', 'DQF': 'DQF_on_pixel'}))
    
#     # add a time stamp
#     dt = pd.Series([pd.to_datetime(ngsinst.ds.attrs['time_coverage_start']), pd.to_datetime(ngsinst.ds.attrs['time_coverage_end'])]).mean().to_datetime64()
#     ds = ds.expand_dims({'datetime': [dt]}, )
    
#     # there was another time coordinate without dimention though ... dropit
#     ds = ds.drop_vars('t')

#     # global attribute
#     ds.attrs['info'] = ('This file contains a projection of satellite data onto SURFRAD sites.\n'
#                          'It includes the closest pixel data as well as the average over circular\n'
#                          'areas with various radii. Note, for the averaged data only data is\n'
#                          'considered with a qulity flag given by the prooduct class in the\n'
#                          'nesdis_gml_synergy package.')
    

#     # save2file
#     ds.to_netcdf(row.path2file_local_processed)
#     # Memory kept on piling up -> maybe a cleanup will help
#     ds.close()
#     ds = None
#     ngsinst.ds.close()
#     ngsinst = None
#     return 

In [13]:
product = 'ACM'
# alt_var = 'BCM'# None, if the varialbe name is not equal to product name
path2processed =  f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/'
query = nesdis_aws.nesdis_aws.AwsQuery(
                                        path2folder_local=f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}/',
                                        satellite='16',
                                        product=f'ABI-L2-{product}',
                                        scan_sector='C',
                                        # start='2020-03-01 18:00:00',
                                        # end='2020-03-01 18:05:00',
                                        #
                                        # CPEX spring 2020
                                        # start='2020-03-01 00:00:00',
                                        # end='2020-06-01 00:00:00',
                                        #
                                        # CPEX original
                                        # start='2018-08-01 00:00:00',
                                        # end='2018-11-01 00:00:00',
                                        #
                                        # all time
                                        start='2017-04-20 13:00:00',
                                        end='2017-04-20 13:05:00',
                                        # end='2022-08-01 00:00:00',
                                        process = dict(#concatenate = 'daily',
                                                       function = lambda row: ngs.projection_function(row, stations), #,, alt_var),
                                                       prefix = f'ABI_L2_{product}_projected2surfrad',
                                                       path2processed = path2processed,
                                                        )
                                        # keep_files=True
                                      )

# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}').mkdir()
# pl.Path(f'/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_{product}_projected2surfrad/').mkdir()

In [14]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed


In [12]:
query.process(raise_exception=True)

In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-31 07:46:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 07:51:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 07:56:18,noaa-goes16/ABI-L2-ACMC/2020/091/07/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 08:01:18,noaa-goes16/ABI-L2-ACMC/2020/091/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-31 08:06:18,noaa-goes16/ABI-L2-ACMC/2020/091/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-20 03:31:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:36:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:41:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:46:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-20 03:51:14,noaa-goes16/ABI-L2-ACMC/2020/080/03/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [9]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-13 08:16:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:21:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:26:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:31:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-13 08:36:14,noaa-goes16/ABI-L2-ACMC/2020/073/08/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [11]:
query.workplan

,path2file_aws,path2file_local,path2file_local_processed
2020-03-07 10:21:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:26:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:31:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:36:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-03-07 10:41:14,noaa-goes16/ABI-L2-ACMC/2020/067/10/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
...,...,...,...
2020-05-31 23:36:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:41:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:46:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...
2020-05-31 23:51:13,noaa-goes16/ABI-L2-ACMC/2020/152/23/OR_ABI-L2-...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...,/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_A...


In [ ]:
24587

In [14]:
ds = xr.open_dataset('/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad/ABI_L2_CTP_projected2surfrad_20200301_180114.nc')

In [15]:
ds

<xarray.Dataset>
Dimensions:                     (datetime: 1, radius: 5, stats: 3,
                                 data_quality: 1, site: 7)
Coordinates:
  * datetime                    (datetime) datetime64[ns] 2020-03-01T18:02:32...
  * radius                      (radius) int64 5 10 25 50 100
  * stats                       (stats) object 'median' 'mean' 'std'
  * site                        (site) object 'BND' 'SXF' 'TBL' ... 'GWN' 'PSU'
  * data_quality                (data_quality) object 'high'
Data variables:
    PRES                        (datetime, data_quality, radius, stats, site) float32 ...
    num_of_valid_points         (datetime, data_quality, radius, site) int64 ...
    PRES_on_pixel               (datetime, site) float32 ...
    DQF_on_pixel                (datetime, site) int64 ...
    argmin_x                    (datetime, site) float32 ...
    argmin_y                    (datetime, site) float32 ...
    lon_gritpoint               (datetime, site) float32 ...
    lat_gridpoint               (datetime, site) float32 ...
    lon_station                 (datetime, site) float32 ...
    lat_station                 (datetime, site) float32 ...
    distance_station_gridpoint  (datetime, site) float32 ...
Attributes:
    info:     This file contains a projection of satellite data onto SURFRAD ...

# Concatenate

In [6]:
reload(ngs)

<module 'nesdis_gml_synergy.satlab' from '/mnt/telg/prog/nesdis_gml_synergy/nesdis_gml_synergy/satlab.py'>

In [10]:
cc = ngs.Concatonator(path2scraped_files='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad/',
                 path2concat_files ='/nfs/stu3data2/Satellite_data/goes/16/ABI_L2_CTP_projected2surfrad_concat/',
                 datetime_format='ABI_L2_CTP_projected2surfrad_%Y%m%d_%H%M%S.nc',
                 test = False,)

In [11]:
cc.workplan

,path2scraped_files,datetime,date,product_name,path2concat_files,p2rf_exists


In [9]:
ds=cc.concat_and_save(save=True, test=False, verbose=False)